<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_PyTorch/blob/main/15CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn

The Cross-Correlation Operation

In [3]:
def corr2d(X, K):
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [4]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

Convolutional Layers

In [5]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [6]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
K = torch.tensor([[1.0, -1.0]])

In [8]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [10]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

Learning a Kernel

In [11]:
# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2). For the sake of simplicity, we ignore the bias here
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

# The two-dimensional convolutional layer uses four-dimensional input and
# output in the format of (example, channel, height, width), where the batch
# size (number of examples in the batch) and the number of channels are both 1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # Learning rate

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # Update the kernel
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

epoch 2, loss 16.628
epoch 4, loss 5.101
epoch 6, loss 1.802
epoch 8, loss 0.690
epoch 10, loss 0.275


In [12]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0397, -0.9326]])

Padding

In [13]:
# We define a helper function to calculate convolutions. It initializes the
# convolutional layer weights and performs corresponding dimensionality
# elevations and reductions on the input and output
def comp_conv2d(conv2d, X):
    # (1, 1) indicates that batch size and the number of channels are both 1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # Strip the first two dimensions: examples and channels
    return Y.reshape(Y.shape[2:])

# 1 row and column is padded on either side, so a total of 2 rows or columns
# are added
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [14]:
# We use a convolution kernel with height 5 and width 3. The padding on either
# side of the height and width are 2 and 1, respectively
conv2d = nn.LazyConv2d(1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

Let's break down how the output shape `(8, 8)` is obtained. The key factors are the input shape, kernel size, and padding. The formula to calculate the output dimensions for a convolutional layer (assuming a stride of 1, which is the default for `nn.Conv2d` when not specified) is:

Output Height = `(Input Height - Kernel Height + 2 * Padding Height) + 1`
Output Width = `(Input Width - Kernel Width + 2 * Padding Width) + 1`

In the code block, you have:
*   **Input shape (X):** `(8, 8)` (so, `Input Height = 8`, `Input Width = 8`)
*   **Kernel size:** `(5, 3)` (so, `Kernel Height = 5`, `Kernel Width = 3`)
*   **Padding:** `(2, 1)` (so, `Padding Height = 2`, `Padding Width = 1`)

Now, let's plug these values into the formulas:

**For the Output Height:**
`Output Height = (8 - 5 + 2 * 2) + 1`
`Output Height = (8 - 5 + 4) + 1`
`Output Height = (3 + 4) + 1`
`Output Height = 7 + 1 = 8`

**For the Output Width:**
`Output Width = (8 - 3 + 2 * 1) + 1`
`Output Width = (8 - 3 + 2) + 1`
`Output Width = (5 + 2) + 1`
`Output Width = 7 + 1 = 8`

Therefore, the resulting output shape is `(8, 8)`, which matches the output `torch.Size([8, 8])` you observed. The padding effectively compensates for the reduction in size caused by the kernel, allowing the output to maintain the same dimensions as the input.

Stride

In [15]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [16]:
conv2d = nn.LazyConv2d(1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

Let's break down how the output shape `(2, 2)` is obtained for the last code block. This time, in addition to input shape, kernel size, and padding, the **stride** plays a crucial role. The general formulas to calculate the output dimensions for a convolutional layer are:

Output Height = `floor((Input Height - Kernel Height + 2 * Padding Height) / Stride Height) + 1`
Output Width = `floor((Input Width - Kernel Width + 2 * Padding Width) / Stride Width) + 1`

In the code block, you have:
*   **Input shape (X):** `(8, 8)` (so, `Input Height = 8`, `Input Width = 8`)
*   **Kernel size:** `(3, 5)` (so, `Kernel Height = 3`, `Kernel Width = 5`)
*   **Padding:** `(0, 1)` (so, `Padding Height = 0`, `Padding Width = 1`)
*   **Stride:** `(3, 4)` (so, `Stride Height = 3`, `Stride Width = 4`)

Now, let's plug these values into the formulas:

**For the Output Height:**
`Output Height = floor((8 - 3 + 2 * 0) / 3) + 1`
`Output Height = floor((5 + 0) / 3) + 1`
`Output Height = floor(5 / 3) + 1`
`Output Height = floor(1.666...) + 1`
`Output Height = 1 + 1 = 2`

**For the Output Width:**
`Output Width = floor((8 - 5 + 2 * 1) / 4) + 1`
`Output Width = floor((3 + 2) / 4) + 1`
`Output Width = floor(5 / 4) + 1`
`Output Width = floor(1.25) + 1`
`Output Width = 1 + 1 = 2`

Therefore, the resulting output shape is `(2, 2)`, which matches the output `torch.Size([2, 2])` you observed. The stride significantly reduces the output dimensions by controlling how many pixels the kernel skips during its movement across the input.